In [5]:
from imutils.object_detection import non_max_suppression
import numpy as np
import argparse
import time
import cv2
 
# construct the argument parser and parse the arguments
width=320
height=320
min_confidence=0.5
modelpath="F:\\software\\frozen_east_text_detection.pb"
imagepath="C:\\Users\\bbx\\new01.png"
# load the input image and grab the image dimensions
image = cv2.imread(imagepath)
orig = image.copy()
(H, W) = image.shape[:2]
 
# set the new width and height and then determine the ratio in change
# for both the width and height
(newW, newH) = (width,height)
rW = W / float(newW)
rH = H / float(newH)
 
# resize the image and grab the new image dimensions
image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]
 
# define the two output layer names for the EAST detector model that
# we are interested -- the first is the output probabilities and the
# second can be used to derive the bounding box coordinates of text
layerNames = [
	"feature_fusion/Conv_7/Sigmoid",
	"feature_fusion/concat_3"]
 
# load the pre-trained EAST text detector
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet(modelpath)
 
# construct a blob from the image and then perform a forward pass of
# the model to obtain the two output layer sets
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
	(123.68, 116.78, 103.94), swapRB=True, crop=False)
start = time.time()
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)
end = time.time()
 
# show timing information on text prediction
print("[INFO] text detection took {:.6f} seconds".format(end - start))
 
# grab the number of rows and columns from the scores volume, then
# initialize our set of bounding box rectangles and corresponding
# confidence scores
(numRows, numCols) = scores.shape[2:4]
rects = []
confidences = []
 
# loop over the number of rows
for y in range(0, numRows):
	# extract the scores (probabilities), followed by the geometrical
	# data used to derive potential bounding box coordinates that
	# surround text
	scoresData = scores[0, 0, y]
	xData0 = geometry[0, 0, y]
	xData1 = geometry[0, 1, y]
	xData2 = geometry[0, 2, y]
	xData3 = geometry[0, 3, y]
	anglesData = geometry[0, 4, y]
 
	# loop over the number of columns
	for x in range(0, numCols):
		# if our score does not have sufficient probability, ignore it
		if scoresData[x] < min_confidence:
			continue
 
		# compute the offset factor as our resulting feature maps will
		# be 4x smaller than the input image
		(offsetX, offsetY) = (x * 4.0, y * 4.0)
 
		# extract the rotation angle for the prediction and then
		# compute the sin and cosine
		angle = anglesData[x]
		cos = np.cos(angle)
		sin = np.sin(angle)
 
		# use the geometry volume to derive the width and height of
		# the bounding box
		h = xData0[x] + xData2[x]
		w = xData1[x] + xData3[x]
 
		# compute both the starting and ending (x, y)-coordinates for
		# the text prediction bounding box
		endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
		endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
		startX = int(endX - w)
		startY = int(endY - h)
 
		# add the bounding box coordinates and probability score to
		# our respective lists
		rects.append((startX, startY, endX, endY))
		confidences.append(scoresData[x])
 
# apply non-maxima suppression to suppress weak, overlapping bounding
# boxes
boxes = non_max_suppression(np.array(rects), probs=confidences)
 
# loop over the bounding boxes
for (startX, startY, endX, endY) in boxes:
	# scale the bounding box coordinates based on the respective
	# ratios
	startX = int(startX * rW)
	startY = int(startY * rH)
	endX = int(endX * rW)
	endY = int(endY * rH)
 
	# draw the bounding box on the image
	cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 0, 255), 2)
 
# show the output image
cv2.imshow("Text Detection", orig)
cv2.waitKey(0)


[INFO] loading EAST text detector...
[INFO] text detection took 1.123064 seconds


-1

In [17]:
##python 图片中查找子图片

import aircv as ac
import pyautogui

def matchImg(imgsrc, imgobj, confidencevalue=0.5):  # imgsrc=原始图像，imgobj=待查找的图片
    imsrc = ac.imread(imgsrc)
    imobj = ac.imread(imgobj)

    match_result = ac.find_template(imsrc, imobj,confidencevalue)  # {'confidence': 0.5435812473297119, 'rectangle': ((394, 384), (394, 416), (450, 384), (450, 416)), 'result': (422.0, 400.0)}
    if match_result is not None:
        match_result['shape'] = (imsrc.shape[1], imsrc.shape[0])  # 0为高，1为宽

    return match_result

imgsrc = "C:\\Users\\bbx\\ggct.png"
imgobj = "F:\\swap\\zlq1\\0059.jpg"
match_result = matchImg(imgsrc, imgobj)
print(match_result["rectangle"][0][0])

318


In [3]:
#coding=utf-8
#图片修复
from PIL import Image
from io import BytesIO 
import cv2
import numpy as np
 
path = "F:\\swap\\zlq1x\\0060.jpg"
 
img = cv2.imread(path)
image = Image.open(path)
r, g, b = image.split()     # 将图片分割成 红绿蓝 三个
r.show()
#g.show()
#b.show() 

hight, width, depth = img.shape[0:3]
 
#图片二值化处理，
lower_hsv = np.array([125, 43, 46])  
upper_hsv = np.array([155, 255, 255]) 
thresh = cv2.inRange(img,lower_hsv,upper_hsv)
 # = cv2.inRange(img, np.array([240, 240, 240]), np.array([255, 255, 255]))
 
#创建形状和尺寸的结构元素
#kernel = np.ones((3, 3), np.uint8)
 
#扩张待修复区域
#hi_mask = cv2.dilate(thresh, kernel, iterations=1)
#specular = cv2.inpaint(img, hi_mask, 5, flags=cv2.INPAINT_TELEA)
 
cv2.namedWindow("Image", 0)
cv2.resizeWindow("Image", int(width / 2), int(hight / 2))
cv2.imshow("Image", thresh)
 
#cv2.namedWindow("newImage", 0)
#cv2.resizeWindow("newImage", int(width / 2), int(hight / 2))
#cv2.imshow("newImage", specular)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# 必须安装pip install opencv-python 和 pillow

import cv2
import numpy as np
from PIL import Image


def drop_wartermark(path, newpath):
    img = cv2.imread(path, 1)
    # img.shape[:3] 则取彩色图片的高、宽、通道。
    hight, width, depth = img.shape[0:3]
    print(hight)
    print(width)
    # 裁剪水印坐标为[y0:y,x0:x1]
    cropped = img # [int(hight * 1):hight, int(width * 1):width]
    # cropped = img[hight-49:hight, width-180:width]
    cv2.imwrite(newpath, cropped)
    # 将图片加载为内存对象 参一：完整路径；参二：flag：-1彩色，0灰色，1原有
    imgsy = cv2.imread(newpath, 1)
    
    # 图片二值化处理，把[200,200,200]~[255, 255, 255]以外的颜色变成0
    # 这个颜色区间就是水印周边的背景颜色
    thresh = cv2.inRange(imgsy, np.array([125, 8, 8]), np.array([155, 25, 25]))
    # #创建形状和尺寸的结构元素 创建水印蒙层
    kernel = np.ones((3, 3), np.uint8)
    # 对水印蒙层进行膨胀操作
    hi_mask = cv2.dilate(thresh, kernel, iterations=10)
    specular = cv2.inpaint(imgsy, hi_mask, 5, flags=cv2.INPAINT_TELEA)
    cv2.imshow("Image", specular)
    cv2.imwrite(newpath, specular)
    
    # 覆盖图片
    imgsy = Image.open(newpath)
    img = Image.open(path)
    img.paste(imgsy, (int(width * 0.7), int(hight * 0.9), width, hight))
    img.save(newpath)
    
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
drop_wartermark("F:\\swap\\zlq1x\\0060.jpg", "newDeWM.png")


1942
1375


ValueError: images do not match

In [13]:
###利用openCV+python进行HSV颜色识别，并结合滑动条动态改变目标颜色
from PIL import Image
import cv2
import numpy as np
#定义窗口名称
winName='Colors of the rainbow'
#定义滑动条回调函数，此处pass用作占位语句保持程序结构的完整性
def nothing(x):
    pass
img_original=cv2.imread('F:\\swap\\zlq1x\\0060.jpg')
#颜色空间的转换
img_hsv=cv2.cvtColor(img_original,cv2.COLOR_BGR2HSV)
#新建窗口
cv2.namedWindow(winName)
#新建6个滑动条，表示颜色范围的上下边界，这里滑动条的初始化位置即为黄色的颜色范围
cv2.createTrackbar('LowerbH',winName,145,255,nothing)
cv2.createTrackbar('LowerbS',winName,4,255,nothing)
cv2.createTrackbar('LowerbV',winName,178,255,nothing)
cv2.createTrackbar('UpperbH',winName,180,255,nothing)
cv2.createTrackbar('UpperbS',winName,13,255,nothing)
cv2.createTrackbar('UpperbV',winName,241,255,nothing)
#while(1):
    #函数cv2.getTrackbarPos()范围当前滑块对应的值
lowerbH=cv2.getTrackbarPos('LowerbH',winName)
lowerbS=cv2.getTrackbarPos('LowerbS',winName)
lowerbV=cv2.getTrackbarPos('LowerbV',winName)
upperbH=cv2.getTrackbarPos('UpperbH',winName)
upperbS=cv2.getTrackbarPos('UpperbS',winName)
upperbV=cv2.getTrackbarPos('UpperbV',winName)
#得到目标颜色的二值图像，用作cv2.bitwise_and()的掩模
img_target=cv2.inRange(img_hsv,(lowerbH,lowerbS,lowerbV),(upperbH,upperbS,upperbV))
#cv2.imshow('tar',img_target)

mask_not = cv2.bitwise_not(img_target)
#cv2.imshow('not',mask_not)

im = Image.open('F:\\swap\\zlq1x\\0060.jpg')
x,y = im.size 
p1 = Image.new('RGBA', im.size, (255,255,255))
p1.save('tmp.png')
img_bg=cv2.imread('tmp.png')

dst = cv2.add(img_bg,img_original)
#plt.imshow(dst[...,::-1])

#输入图像与输入图像在掩模条件下按位与，得到掩模范围内的原图像
img_specifiedColor=cv2.bitwise_and(img_bg,img_bg,mask=img_target)
cv2.imwrite('temp001.jpg',img_specifiedColor)
img2_fg = cv2.imread('temp001.jpg')
dst = cv2.add(img_original,img2_fg)

cv2.imwrite('060.jpg',dst)
#img_specifiedColor=cv2.bitwise_and(img_specifiedColor,img_original,mask=mask_not)
cv2.namedWindow(winName, cv2.WINDOW_GUI_EXPANDED)
#cv2.resizeWindow(winName,1200, 1900)    
cv2.imshow(winName,dst) #img_specifiedColor)
#    if cv2.waitKey(1)==ord('q'):
#        break
cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
logo1="""
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                            ┃
┃ 去除图片中的广告小方块 1.0         作者 kingboy                                                            ┃
┃ PYTHON 得到光标处的句柄,控件标题,控件窗口大小,进程名称及ID                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

┌┍┎┏┐┑┒┓└┕┖┗┘┙┚┛┕└┖
"""
# 实现查找广告图片并擦除后保存到out目录,20210616

from PIL import Image, ImageDraw,ImageFont
import aircv as ac
import pyautogui
w1 = 1
h1 = 1
def matchImg(imgsrc, imgobj, confidencevalue=0.5):  # imgsrc=原始图像，imgobj=待查找的图片
    imsrc = ac.imread(imgsrc)
    imobj = ac.imread(imgobj)
    global w1,h1
    h1 = (imsrc.shape[0])
    w1 = (imsrc.shape[1])
    match_result = None
    if imsrc.shape[0] < imobj.shape[0] and imsrc.shape[1] < imobj.shape[1] :
        match_result = ac.find_template(imsrc, imobj,confidencevalue)  # {'confidence': 0.5435812473297119, 'rectangle': ((394, 384), (394, 416), (450, 384), (450, 416)), 'result': (422.0, 400.0)}
    if match_result is not None:
        match_result['shape'] = (imsrc.shape[1], imsrc.shape[0])  # 0为高，1为宽
        print('v',end = '. ')

    #print(match_result)
    return match_result


#h1 = 1 #目标高度
#w1 = 1 #目标宽度

imgsrc = "C:\\Users\\bbx\\ggjt.png"
imgsrc2 = "C:\\Users\\bbx\\ggct.png"

for num in range(2,257):
    print(num,end=",")#("F:\\swap\\zlq1\\out\\"+  "{0:0>4}".format(num)  +".jpg")
    imgobj = "F:\\swap\\zlq1x\\"+  "{0:0>4}".format(num)  +".jpg"

    #print(w1,h1)
    #print(match_result["rectangle"])
    #print(match_result["rectangle"][0][0], match_result["rectangle"][0][1], match_result["rectangle"][1][0], match_result["rectangle"][1][1])

    Image1 = Image.open(imgobj)
    draw = ImageDraw.Draw(Image1)
    #draw.line((20, 20, 150, 150), 'cyan')
    
    match_result = matchImg(imgsrc, imgobj)
    #print(match_result)
    if match_result is not None :
        draw.rectangle((match_result["rectangle"][0][0], match_result["rectangle"][0][1], match_result["rectangle"][0][0]+w1, match_result["rectangle"][0][1]+h1), 'white', 'white')
    else:
        pass
    
    #match_result2 =   matchImg(imgsrc2, imgobj)
    #if match_result2 is not None :
    #    draw.rectangle((match_result2["rectangle"][0][0], match_result2["rectangle"][0][1], match_result2["rectangle"][0][0]+w1, match_result2["rectangle"][0][1]+h1), 'white', 'white')
    #else :
    #    pass
    
    Image1.save("F:\\swap\\zlq1x\\out\\"+  "{0:0>4}".format(num)  +".jpg", "JPEG")
#draw.rectangle((101, 190, 301, 395), 'yellow', 'black')
#draw.arc((100, 200, 300, 400), 0, 180, 'yellow')
#draw.ellipse((350, 300, 500, 400), 'yellowgreen', 'wheat')

#font = ImageFont.truetype("consola.ttf", 40, encoding="unic")#设置字
#draw.text((100, 50), u'Hello World', 'fuchsia', font)

#draw.line((0,0) +Image1.size, fill=128)
#Image1.show()



2,3,v. 4,v. 5,v. 6,v. 7,v. 8,v. 9,v. 10,v. 11,v. 12,v. 13,v. 14,v. 15,v. 16,v. 17,v. 18,v. 19,v. 20,21,v. 22,v. 23,v. 24,v. 25,v. 26,v. 27,v. 28,v. 29,v. 30,31,v. 32,v. 33,v. 34,v. 35,v. 36,v. 37,v. 38,v. 39,v. 40,v. 41,v. 42,v. 43,v. 44,v. 45,v. 46,v. 47,v. 48,v. 49,v. 50,51,v. 52,v. 53,v. 54,v. 55,v. 56,v. 57,v. 58,v. 59,v. 60,v. 61,v. 62,v. 63,v. 64,v. 65,v. 66,v. 67,v. 68,v. 69,v. 70,v. 71,v. 72,v. 73,v. 74,v. 75,v. 76,v. 77,v. 78,v. 79,v. 80,81,v. 82,v. 83,v. 84,v. 85,v. 86,v. 87,v. 88,v. 89,v. 90,v. 91,v. 92,v. 93,v. 94,v. 95,v. 96,v. 97,v. 98,v. 99,v. 100,v. 101,v. 102,v. 103,v. 104,v. 105,v. 106,v. 107,v. 108,v. 109,v. 110,v. 111,v. 112,v. 113,v. 114,v. 115,v. 116,v. 117,v. 118,v. 119,v. 120,121,v. 122,v. 123,v. 124,v. 125,v. 126,v. 127,v. 128,v. 129,v. 130,v. 131,v. 132,v. 133,v. 134,v. 135,v. 136,v. 137,v. 138,v. 139,v. 140,v. 141,v. 142,v. 143,v. 144,v. 145,v. 146,v. 147,v. 148,v. 149,v. 150,v. 151,v. 152,v. 153,v. 154,v. 155,v. 156,v. 157,v. 158,v. 159,v. 160,161,v. 162,v. 

In [ ]:
"{0:0>4}".format(11)

In [7]:
logo1="""
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                            ┃
┃ 去除黑白文字下面的粉色水印 1.0     作者 kingboy                                                            ┃
┃ PYTHON 得到光标处的句柄,控件标题,控件窗口大小,进程名称及ID                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

┌┍┎┏┐┑┒┓└┕┖┗┘┙┚┛┕└┖
"""
#去除黑白文字下面的粉色水印

import cv2
import numpy as np


#定义滑动条回调函数，此处pass用作占位语句保持程序结构的完整性
def nothing(x):
    pass

for num in range(256,257):
    pagepath = "F:\\swap\\zlq1x\\out\\"+  "{0:0>4}".format(num)  +".jpg"
    outpath  =  "F:\\swap\\zlq1x\\out\\out2\\"+  "{0:0>4}".format(num)  +".jpg"
    img_original=cv2.imread(pagepath) #用来作原图用
    img_book  = cv2.imread(pagepath)   # 用来作选取蒙板区域用
    img_mask  = cv2.imread('bookmask.bmp',-1) #用来作蒙板遮罩
    img_white = cv2.imread('bookmask.bmp') #用来作替换的白色

    #cv2.imshow('msk',img_mask)
    #print(img_book.shape,img_mask.shape)
    if num in [2,20,30,50,80,120,160,200,250] :   #应对尺寸不规整的图片,直接忽略复制保存到目的文件夹.
        cv2.imwrite(outpath, img_book)
        continue
    else:    
        #pass #单独使用动态调整时使用
        img_book = cv2.bitwise_and(img_book,img_book,mask=img_mask) #擦除水印时使用


    #print(img_original.shape,img_book.shape)
    #颜色空间的转换
    img_hsv=cv2.cvtColor(img_book,cv2.COLOR_BGR2HSV)


     #需要动态调整时解开此块注释
    #新建6个滑动条，表示颜色范围的上下边界，这里滑动条的初始化位置即为黄色的颜色范围
    winName=r'通道选择 hsv模式' #定义窗口名称
    cv2.namedWindow(winName,0)      #新建窗口
    cv2.createTrackbar('LowerbH',winName,135,255,nothing)
    cv2.createTrackbar('LowerbS',winName,2,255,nothing)
    cv2.createTrackbar('LowerbV',winName,184,255,nothing)
    cv2.createTrackbar('UpperbH',winName,173,255,nothing)
    cv2.createTrackbar('UpperbS',winName,82,255,nothing)
    cv2.createTrackbar('UpperbV',winName,255,255,nothing)
    while(1):
        #函数cv2.getTrackbarPos()范围当前滑块对应的值
        lowerbH=cv2.getTrackbarPos('LowerbH',winName)
        lowerbS=cv2.getTrackbarPos('LowerbS',winName)
        lowerbV=cv2.getTrackbarPos('LowerbV',winName)
        upperbH=cv2.getTrackbarPos('UpperbH',winName)
        upperbS=cv2.getTrackbarPos('UpperbS',winName)
        upperbV=cv2.getTrackbarPos('UpperbV',winName)
        #得到目标颜色的二值图像，用作cv2.bitwise_and()的掩模
        img_target = cv2.inRange(img_hsv,(lowerbH,lowerbS,lowerbV),(upperbH,upperbS,upperbV))
        #输入图像与输入图像在掩模条件下按位与，得到掩模范围内的原图像
        #cv2.imshow('tar',img_target)
        img_specifiedColor=cv2.bitwise_and(img_white,img_white,mask=img_target)
        #cv2.imshow('demo',img_specifiedColor)
        dst = cv2.add(img_original,img_specifiedColor)
        cv2.imshow("p"+str(num),dst)
        if cv2.waitKey(1)==ord('q'):
            break
    cv2.destroyAllWindows()
    
    #需要动态调整时解开以上块注释
    

    #  动态调整时注释掉以下两行
    #img_target = cv2.inRange(img_hsv,(135,2,184),(173,82,255))
    #img_specifiedColor=cv2.bitwise_and(img_white,img_white,mask=img_target)

    #  动态调整时注释掉以上两行 

    dst = cv2.add(img_original,img_specifiedColor)    
    cv2.imwrite(outpath, dst)
    print(num,end=",")
    

#cv2.imshow('new1',dst)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

#cv2.imwrite("bookmask.png", img_target)

256,

In [28]:
for num in range(2,257):
    pagepath = "F:\\swap\\zlq1x\\out\\"+  "{0:0>4}".format(num)  +".jpg"
    img1=cv2.imread(pagepath) #用来作原图用
    if not (img1.shape[0] == 1942 and img1.shape[1] == 1375) :
        #print(img1.shape,end="")
        print(num,img1.shape[0],img1.shape[1],end=".")

2 1942 1361.20 3088 2182.30 3088 2182.50 3088 2182.80 3088 2182.120 3088 2182.160 3076 2174.200 3088 2182.250 3088 2182.

In [1]:
import cv2
import numpy as np


#定义滑动条回调函数，此处pass用作占位语句保持程序结构的完整性
def nothing(x):
    pass

for num in range(301,302):
    pagepath = "F:\\swap\\zlq1x\\out\\"+  "{0:0>4}".format(num)  +".jpg"
    outpath  =  "F:\\swap\\zlq1x\\out\\out2\\"+  "{0:0>4}".format(num)  +".jpg"
    img_original=cv2.imread(pagepath) #用来作原图用
    img_book  = cv2.imread(pagepath)   # 用来作选取蒙板区域用

    #颜色空间的转换
    img_hsv=cv2.cvtColor(img_book,cv2.COLOR_BGR2HSV)


     #需要动态调整时解开此块注释
    #新建6个滑动条，表示颜色范围的上下边界，这里滑动条的初始化位置即为黄色的颜色范围
    winName=r'通道选择 hsv模式' #定义窗口名称
    cv2.namedWindow(winName,0)      #新建窗口
    cv2.createTrackbar('LowerbH',winName,135,255,nothing)
    cv2.createTrackbar('LowerbS',winName,2,255,nothing)
    cv2.createTrackbar('LowerbV',winName,184,255,nothing)
    cv2.createTrackbar('UpperbH',winName,173,255,nothing)
    cv2.createTrackbar('UpperbS',winName,82,255,nothing)
    cv2.createTrackbar('UpperbV',winName,255,255,nothing)
    while(1):
        #函数cv2.getTrackbarPos()范围当前滑块对应的值
        lowerbH=cv2.getTrackbarPos('LowerbH',winName)
        lowerbS=cv2.getTrackbarPos('LowerbS',winName)
        lowerbV=cv2.getTrackbarPos('LowerbV',winName)
        upperbH=cv2.getTrackbarPos('UpperbH',winName)
        upperbS=cv2.getTrackbarPos('UpperbS',winName)
        upperbV=cv2.getTrackbarPos('UpperbV',winName)
        #得到目标颜色的二值图像，用作cv2.bitwise_and()的掩模
        img_target = cv2.inRange(img_hsv,(lowerbH,lowerbS,lowerbV),(upperbH,upperbS,upperbV))
        #输入图像与输入图像在掩模条件下按位与，得到掩模范围内的原图像
        #cv2.imshow('tar',img_target)
        #img_specifiedColor=cv2.bitwise_and(img_white,img_white,mask=img_target)
        #cv2.imshow('demo',img_specifiedColor)
        #dst = cv2.add(img_original,img_target)
        cv2.imshow("p"+str(num),img_target)
        if cv2.waitKey(1)==ord('q'):
            break
    cv2.destroyAllWindows()
    
    #需要动态调整时解开以上块注释
    

    #  动态调整时注释掉以下两行
    #img_target = cv2.inRange(img_hsv,(135,2,184),(173,82,255))
    #img_specifiedColor=cv2.bitwise_and(img_white,img_white,mask=img_target)

    #  动态调整时注释掉以上两行 

    dst = cv2.add(img_original,img_specifiedColor)    
    cv2.imwrite(outpath, dst)
    print(num,end=",")

NameError: name 'img_specifiedColor' is not defined

In [9]:
oath = '我爱妞'
print(type(oath))
print(len(oath))
 
oath1 = u'我爱妞'
print(type(oath1))
print(len(oath1))
 
print(oath==oath1)
 
 
utf8 = oath.encode('utf-8')
print(type(utf8))
print(len(utf8))
print(utf8)
 
gbk = oath.encode('gbk')
print(type(gbk))
print(len(gbk))
print(gbk)
 
 
out = open('test.txt','w',encoding = 'utf-8')
 
test = u'\u5220\u9664'
print(len(test))
print(test)
test1 = test.encode('utf-8')
print(test1)
print(type(test1))
 
out.write(test)
out.close()


<class 'str'>
3
<class 'str'>
3
True
<class 'bytes'>
9
b'\xe6\x88\x91\xe7\x88\xb1\xe5\xa6\x9e'
<class 'bytes'>
6
b'\xce\xd2\xb0\xae\xe6\xa4'
2
删除
b'\xe5\x88\xa0\xe9\x99\xa4'
<class 'bytes'>


In [ ]:
#####
#1,获取区域
#2,遮罩